<a href="https://colab.research.google.com/github/HilbertN/Redes_Neuronales/blob/main/RN16_ResidualConv2DBlocks2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
learning_rate = 0.01
epochs = 30
batch_size = 120
optimizer = 'rmsprop'

In [ ]:
!wget https://dagshub.com/jorgevc/Gatos-y-Perros/raw/24eaabd8ca81ed89f780138c1e3955ccd1f3d155/data/DogsCats.zip
!unzip DogsCats.zip

In [ ]:
###################
#### Load dataset ####
ih, iw = 150, 150 #tamano de la imagen
input_shape = (ih, iw, 3) #forma de la imagen: alto ancho y numero de canales

#train_dir = 'data/minitrain' #directorio de entrenamiento
#test_dir = 'data/minitest' #directorio de prueba
train_dir = 'train' #directorio de entrenamiento
test_dir = 'test' #directorio de prueba


num_class = 2 #cuantas clases
epochs = 30 #cuantas veces entrenar. En cada epoch hace una mejora en los parametros

batch_size = 50 #batch para hacer cada entrenamiento. Lee 50 'batch_size' imagenes antes de actualizar los parametros. Las carga a memoria
num_train = 20000 #numero de imagenes en train
num_test = 5000 #numero de imagenes en test


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


gentrain = ImageDataGenerator() #indica que reescale cada canal con valor entre 0 y 1.


train = gentrain.flow_from_directory(train_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

gentest = ImageDataGenerator()

test = gentest.flow_from_directory(test_dir,
                batch_size=batch_size,
                target_size=(iw, ih),
                class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
#### Define Model ####

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

def block(x,base_filter=32, pooling=True):
    residual = x
    x = layers.Conv2D(base_filter, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(2*base_filter, 3, activation="relu", padding="same")(x)
    if (pooling == True):
        x = layers.MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(2*base_filter, 1, strides=2)(residual)
    else:
        residual = layers.Conv2D(2*base_filter, 1)(residual)
    x = layers.add([x, residual])
    return x

x = block(x,base_filter=32)
x = block(x,base_filter=48)
x = block(x,base_filter=64,pooling=False)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

######

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 32, 32, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 32)           896       ['rescaling[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)           (None, 32, 32, 64)           18496     ['conv2d[0][0]']              
                                                                                              

In [ ]:

############ Train model ###

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=test
    )

model.save("Conv2D_2.hf5")


Epoch 1/30
400/400 [==============================] - 3814s 10s/step - loss: 0.6745 - accuracy: 0.5669 - val_loss: 0.6628 - val_accuracy: 0.5968
Epoch 2/30
181/400 [============>.................] - ETA: 30:51 - loss: 0.6468 - accuracy: 0.6124